In [13]:
# --- Fetch MP elemental reference energies for Li, La, Zr, O (eV/atom) ---
# pip install pymatgen

from pymatgen.core import Element
from pymatgen.ext.matproj import MPRester
from pymatgen.analysis.phase_diagram import PhaseDiagram

# <<< CHANGE YOUR API KEY HERE >>>
MP_API_KEY = "j3J85pX4nLw6asHG9E2lbbCHEKDKgrjc"   # ← edit this line anytime

def mp_elemental_reference_energies(elements=("Li", "La", "Zr", "O"), api_key=MP_API_KEY):
    """Return MP elemental reference energies (eV/atom) as a dict {symbol: mu_ref}."""
    refs = {}
    with MPRester(api_key) as mpr:
        for sym in elements:
            print(f"Fetching reference for {sym} ...")
            entries = mpr.get_entries(sym, inc_structure=False)
            if not entries:
                raise RuntimeError(f"No entries found for {sym}")
            pd = PhaseDiagram(entries)
            mu = pd.el_refs[Element(sym)].energy_per_atom
            refs[sym] = mu
    return refs


if __name__ == "__main__":
    refs = mp_elemental_reference_energies(("Li", "La", "Zr", "O"))
    print("\n--- MP Elemental Reference Energies ---")
    for k, v in refs.items():
        print(f"{k}: {v:.6f} eV/atom")


Fetching reference for Li ...


/tmp/ipykernel_3135369/1173337282.py:17: DeprecationWarning: The inc_structure, conventional_unit_cell, and sort_by_e_above_hull arguments are deprecated. These arguments have no effect and will be removed in 2026.1.1.
  entries = mpr.get_entries(sym, inc_structure=False)


Fetching reference for La ...


/tmp/ipykernel_3135369/1173337282.py:17: DeprecationWarning: The inc_structure, conventional_unit_cell, and sort_by_e_above_hull arguments are deprecated. These arguments have no effect and will be removed in 2026.1.1.
  entries = mpr.get_entries(sym, inc_structure=False)


Fetching reference for Zr ...


/tmp/ipykernel_3135369/1173337282.py:17: DeprecationWarning: The inc_structure, conventional_unit_cell, and sort_by_e_above_hull arguments are deprecated. These arguments have no effect and will be removed in 2026.1.1.
  entries = mpr.get_entries(sym, inc_structure=False)


Fetching reference for O ...


/tmp/ipykernel_3135369/1173337282.py:17: DeprecationWarning: The inc_structure, conventional_unit_cell, and sort_by_e_above_hull arguments are deprecated. These arguments have no effect and will be removed in 2026.1.1.
  entries = mpr.get_entries(sym, inc_structure=False)



--- MP Elemental Reference Energies ---
Li: -1.908923 eV/atom
La: -4.936007 eV/atom
Zr: -8.547701 eV/atom
O: -4.947961 eV/atom


In [ ]:
import os
from pymatgen.core import Structure
from chgnet.model import CHGNet

# ---- 1. CHGNet model -------------------------------------------------
model = CHGNet.load()   # e_pred is eV/atom !

# ---- 2. elemental chemical potentials (CHGNet column) ----------------
mu = {"Li": -1.908923, "La": -4.936007, "Zr": -8.547701, "O": -4.947691}

# ---- 3. CIFs ----------------------------------------------------------
cif_dir = "/home/phanim/harshitrawat/summer/llzo_sanity_check_srinibas"
files = {
    "LLZO_Srinibas.cif": "LLZO"
}

for fname, label in files.items():
    struct = Structure.from_file(os.path.join(cif_dir, fname))
    n_atoms = struct.composition.num_atoms
    
    # ------- CHGNet prediction (already per atom) ----------------------
    e_pred_atom = model.predict_structure(struct)["e"]
    
    # ------- reference energy per atom ---------------------------------
    ref_per_atom = sum(struct.composition[el] * mu[str(el)] for el in struct.elements) / n_atoms
    
    # ------- formation energy per atom ---------------------------------
    e_form = e_pred_atom - ref_per_atom
    
    print(f"{label:15s}:  E_form (CHGNet) = {e_form: .6f} eV/atom")
    # print total energy
    print(f"{label:15s}:  E_tot  (CHGNet) = {e_pred_atom * n_atoms: .6f} eV")
    

/home/phanim/harshitrawat/miniconda3/envs/mace_0.3.8/lib/python3.10/site-packages/chgnet/model/model.py:673: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load

CHGNet v0.3.0 initialized with 412,525 parameters
CHGNet will run on cpu
LLZO           :  E_form (CHGNet) =  1.760684 eV/atom
LLZO           :  E_tot  (CHGNet) = -470.290833 eV


In [24]:
import os
from pymatgen.core import Structure
from chgnet.model import CHGNet

# ---- 1. CHGNet model -------------------------------------------------
model = CHGNet.load()   # e_pred is eV/atom !

# ---- 2. elemental chemical potentials (CHGNet column) ----------------
mu = {"Li": -1.882, "La": -4.894, "Zr": -8.509, "O": -4.913}

# ---- 3. CIFs ----------------------------------------------------------
cif_dir = "/home/phanim/harshitrawat/summer/formation_energy/cifs"
files = {
    "mp-942733.cif": "LLZO"
}

for fname, label in files.items():
    struct = Structure.from_file(os.path.join(cif_dir, fname))
    n_atoms = struct.composition.num_atoms
    
    # ------- CHGNet prediction (already per atom) ----------------------
    e_pred_atom = model.predict_structure(struct)["e"]
    
    # ------- reference energy per atom ---------------------------------
    ref_per_atom = sum(struct.composition[el] * mu[el.symbol] for el in struct.elements) / n_atoms
    
    # ------- formation energy per atom ---------------------------------
    e_form = e_pred_atom - ref_per_atom
    
    print(f"{label:15s}:  E_form (CHGNet) = {e_form: .6f} eV/atom")
    # print total energy
    print(f"{label:15s}:  E_tot  (CHGNet) = {e_pred_atom * n_atoms: .6f} eV")
    

CHGNet v0.3.0 initialized with 412,525 parameters
CHGNet will run on cpu
LLZO           :  E_form (CHGNet) = -3.121806 eV/atom
LLZO           :  E_tot  (CHGNet) = -1430.026794 eV


In [1]:
from pymatgen.io.ase import AseAtomsAdaptor
from pymatgen.core import Structure
from chgnet.model import CHGNet
from chgnet.model.dynamics import CHGNetCalculator

chgnet_model = CHGNet.load()

# Create the ASE calculator using the CHGNet model
calculator = CHGNetCalculator(chgnet_model)
adaptor = AseAtomsAdaptor()

pmg_structure = Structure.from_file("/home/phanim/harshitrawat/summer/llzo_sanity_check_srinibas/Li_Srinibas.cif") 
ase_atoms = adaptor.get_atoms(pmg_structure)
ase_atoms.calc = calculator
total_energy = ase_atoms.get_potential_energy()
mu_model_Li = total_energy / len(ase_atoms)
print(f"Li: μ_model = {mu_model_Li:.6f} eV/atom")
# # Let us do this for La, Zr, and O as well
pmg_structure = Structure.from_file("/home/phanim/harshitrawat/summer/llzo_sanity_check_srinibas/La_Srinibas.cif")
ase_atoms = adaptor.get_atoms(pmg_structure)
ase_atoms.calc = calculator
total_energy = ase_atoms.get_potential_energy()
mu_model_La = total_energy / len(ase_atoms)
print(f"La: μ_model = {mu_model_La:.6f} eV/atom")
pmg_structure = Structure.from_file("/home/phanim/harshitrawat/summer/llzo_sanity_check_srinibas/Zr_Srinibas.cif")
ase_atoms = adaptor.get_atoms(pmg_structure)
ase_atoms.calc = calculator
total_energy = ase_atoms.get_potential_energy()
mu_model_Zr = total_energy / len(ase_atoms)
print(f"Zr: μ_model = {mu_model_Zr:.6f} eV/atom")



/home/phanim/harshitrawat/miniconda3/envs/mace_0.3.8/lib/python3.10/site-packages/chgnet/model/model.py:673: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load

CHGNet v0.3.0 initialized with 412,525 parameters
CHGNet will run on cpu
CHGNet will run on cpu
Li: μ_model = -1.875548 eV/atom


/home/phanim/harshitrawat/miniconda3/envs/mace_0.3.8/lib/python3.10/site-packages/pymatgen/core/structure.py:3107: UserWarning: Issues encountered while parsing CIF: 16 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  struct = parser.parse_structures(primitive=primitive)[0]


La: μ_model = -4.893149 eV/atom


/home/phanim/harshitrawat/miniconda3/envs/mace_0.3.8/lib/python3.10/site-packages/pymatgen/core/structure.py:3107: UserWarning: Issues encountered while parsing CIF: 12 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  struct = parser.parse_structures(primitive=primitive)[0]


Zr: μ_model = -8.508489 eV/atom


In [ ]:
pmg_structure = Structure.from_file("/home/phanim/harshitrawat/summer/llzo_sanity_check_srinibas/O2_final_Srinibas.cif")  # Needs to be a periodic solid O2 structure
ase_atoms = adaptor.get_atoms(pmg_structure)
ase_atoms.calc = calculator
total_energy = ase_atoms.get_potential_energy()
mu_model_O = total_energy / len(ase_atoms)
print(f"O: μ_model = {mu_model_O:.6f} eV/atom")


O: μ_model = -4.808473 eV/atom


In [6]:
import os
from pymatgen.core import Structure
from chgnet.model import CHGNet

# ---- 1. CHGNet model -------------------------------------------------
model = CHGNet.load()   # e_pred is eV/atom !

# ---- 2. elemental chemical potentials (CHGNet column) ----------------
mu = {"Li": -1.875548, "La": -4.893149, "Zr": -8.508489, "O": -4.808473}

# ---- 3. CIFs ----------------------------------------------------------
cif_dir = "/home/phanim/harshitrawat/summer/formation_energy/cifs"
files = {
    "mp-942733.cif": "LLZO"
}

for fname, label in files.items():
    struct = Structure.from_file(os.path.join(cif_dir, fname))
    n_atoms = struct.composition.num_atoms
    
    # ------- CHGNet prediction (already per atom) ----------------------
    e_pred_atom = model.predict_structure(struct)["e"]
    
    # ------- reference energy per atom ---------------------------------
    ref_per_atom = sum(struct.composition[el] * mu[el.symbol] for el in struct.elements) / n_atoms
    
    # ------- formation energy per atom ---------------------------------
    e_form = e_pred_atom - ref_per_atom
    
    print(f"{label:15s}:  E_form (CHGNet) = {e_form: .6f} eV/atom")
    # print total energy
    

/home/phanim/harshitrawat/miniconda3/envs/mace_0.3.8/lib/python3.10/site-packages/chgnet/model/model.py:673: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load

CHGNet v0.3.0 initialized with 412,525 parameters
CHGNet will run on cpu
LLZO           :  E_form (CHGNet) = -3.176101 eV/atom
